In [1]:
!ls /keyakizaka_mining/

data  model  result  secret  tools


In [2]:
import argparse
import sys
import os

import datetime
import numpy as np
import cv2

# Parameters

PADDING = 0
CONF_THRESHOLD = 0.5
NMS_THRESHOLD = 0.4
IMG_WIDTH = 416
IMG_HEIGHT = 416

# Default colors
COLOR_BLUE = (255, 0, 0)
COLOR_GREEN = (0, 255, 0)
COLOR_RED = (0, 0, 255)
COLOR_WHITE = (255, 255, 255)
COLOR_YELLOW = (0, 255, 255)

# path

model_cfg = './cfg/yolov3-face.cfg'
model_weights = '/keyakizaka_mining/model/yoloface/model-weights/yolov3-wider_16000.weights'

image_dir = '/keyakizaka_mining/data/images/'
face_dir = '/keyakizaka_mining/data/faceimages_pad%d'%(PADDING)

In [3]:
# functions

def get_outputs_names(net):
    # Get the names of all the layers in the network
    layers_names = net.getLayerNames()
    return [layers_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

def refined_box(left, top, width, height):
    right = left + width
    bottom = top + height

    original_vert_height = bottom - top
    top = int(top + original_vert_height * 0.15)
    bottom = int(bottom - original_vert_height * 0.05)

    margin = ((bottom - top) - (right - left)) // 2
    left = left - margin if (bottom - top - right + left) % 2 == 0 else left - margin - 1
    right = right + margin

    # padding
    left -= PADDING
    top -= PADDING
    right += PADDING
    bottom += PADDING
    
    return left, top, right, bottom

def post_process(image_name, save_face_dir, frame, outs, conf_threshold, nms_threshold):
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only the ones with high confidence scores.
    # Assign the box's class label as the class with the highest score.
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    for i in indices:
        box = boxes[i[0]]
        left, top, width, height = box[0], box[1], box[2], box[3]     
        if left < 0 or top < 0: continue
        if left + width > frame.shape[1] or top + height > frame.shape[0]: continue
        left, top, right, bottom = refined_box(left, top, width, height)
        # cv2.imwrite( 'facetest_%d.png' % i[0], frame[top:bottom,left:right]) # save rectangle
        cv2.imwrite( '%s/face%d_%s.png' % (save_face_dir, i[0], image_name), frame[top:bottom,left:right]) # save rectangle
    return

def save_detected_faceimages( image_path, image_name, save_face_dir ):
    cap = cv2.VideoCapture( image_path )

    has_frame, frame = cap.read()
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT), [0, 0, 0], 1, crop=False) # Create a 4D blob from a frame.

    net.setInput(blob) # Sets the input to the network
    outs = net.forward(get_outputs_names(net)) # Runs the forward pass to get output of the output layers

    post_process(image_name, save_face_dir, frame, outs, CONF_THRESHOLD, NMS_THRESHOLD) # Remove the bounding boxes with low confidence

    cap.release()
    return

In [4]:
# Give the configuration and weight files for the model and load the network using them.

net = cv2.dnn.readNetFromDarknet(model_cfg , model_weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
for member in os.listdir( image_dir ):
    
    print( member )
    
    blog_image_path = '%s/%s' % ( image_dir, member )
    save_face_dir = '%s/%s' % ( face_dir, member )
    if not os.path.exists( save_face_dir ): os.mkdir( save_face_dir )
    
    blog_images = os.listdir( blog_image_path )
    for filename in blog_images:
        image_path = '%s/%s' % ( blog_image_path, filename )
        image_name = filename.split('.')[0] # ...
        
        save_detected_faceimages( image_path, image_name, save_face_dir )